# `pompy` - Puff-based Odour plume Model in  PYthon

`pompy` is a NumPy based implementation of the puff-based odour plume model described in

> Farrell, J.A., Murlis, J., Long, X., Li, W. and Cardé, R.T., 2002. 
> Filament-based atmospheric dispersion model to achieve short time-scale 
> structure of odor plumes. *Environmental fluid mechanics*, 2(1-2), pp.143-169.
> [springer.com](https://link.springer.com/article/10.1023%2FA%3A1016283702837#page-1)

This notebook gives an example of using the `pompy` Python package to generate an animation of an odour plume using the simulation parameters described in the article


## `pompy` modules

The two key modules in the `pompy` package are the `models` and `processors` modules. The `models` module defines classes implementing the simulation of the wind velocity field and odour plume as well as associated utility classes. The `processors` module defines various helper functions for processing the simulated outputs of the model. `pompy` uses [NumPy](http://www.numpy.org/) to perform the vectorised array based calculations within the model simulation and output processing code, and [Matplotlib](https://matplotlib.org/) is used to visualise the processed outputs. We perform the necessary imports in the cell below and set up Matplotlib to allow inline display of plots and animations.

In [1]:
from pompy import models, processors
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib inline
plt.rcParams['animation.html'] = 'html5'

## Random number generator

We first seed a NumPy random number generator object to ensure reproducible results.

In [2]:
seed = 20180517
rng = np.random.RandomState(seed)

## Wind model

A wind velocity model is used to calculate calculate advective transport of the plume. A 2D approximation is used as described in Farrell et al. (2002), with the wind velocities calculated over a regular 2D grid of points using a finite difference method. The boundary conditions at the edges of the simulated region are for both components of the velocity field constant mean values plus coloured noise which drives the meandering of the velocity field. The two velocity field components are calculated for the four corners of the simulated region and then linearly interpolated over the edges.

The wind model is implemented in the `pompy.models.WindModel` class with the following correspondence between parameters in the `WindModel` constructor and the quantities defined in Farrel et al. (2002). In some cases the paper gives a range of values, indicated by square brackets $[\alpha, \beta]$. Farrell et al. (2002) gives two settings for the wind model noise - *small amplitude meander* and *large amplitude meander*, with the curly braces in the table giving the corresponding values in that order.

| `WindModel` parameter    | Farrell et al. symbol | Farrell et al. value       |
|--------------------------|-----------------------|----------------------------|
| `u_av`                   | $\bar{u}(0)$          | 1 m / s                    |
| `v_av`                   | $\bar{v}(0)$          | 0 m / s                    |
| `k_x`                    | $k_x$                 | [1, 30] m² / s             |
| `k_y`                    | $k_y$                 | [1, 30] m² / s             |
| `noise_gain`             | $G$                   | {3, 20}                    |
| `noise_damp`             | $\frac{b}{2\sqrt{a}}$ | {0.071, 0.1}               |
| `noise_bandwidth`        | $\sqrt{a}$            | {0.71, 0.2}                |

**Note**: `pompy` uses a different parameterisation for the coloured noise model in terms of damping coefficient $\zeta$ and bandwidth $\omega$ parameters rather than the transfer function coefficients $a$ and $b$ used in Farrell et al. (2002) however there is a one-to-one correspondence via $\zeta = \frac{b}{2\sqrt{a}}$ and $\omega = \sqrt{a}$.

In the cell below we define a dictionary of wind model parameters as in the above table, using *large amplitude meander* values for coloured noise model. We use a value of 10 for $k_x$ and $k_y$ which is roughly in the middle of the Farrell et al. (2002) recommended interval (and consistent with the value used in the [code](https://intra.ece.ucr.edu/~Farrell/?page=content/CPT.html) accompanying the article).

In [3]:
wind_model_params = {
    'u_av': 1.,  # Mean x-component of wind velocity (u).
    'v_av': 0.,  # Mean y-component of wind velocity (v).
    'k_x': 10.,  # Diffusivity constant in x direction.
    'k_y': 10.,  # Diffusivity constant in y direction.
    'noise_gain': 20.,  # Input gain constant for boundary condition noise generation.
    'noise_damp': 0.1,  # Damping ratio for boundary condition noise generation.
    'noise_bandwidth': 0.2,  # Bandwidth for boundary condition noise generation.
}

The code accompanying the Farrell et al. (2002) article, uses a coloured noise generation scheme with update of the form

$$
  \begin{bmatrix} x_{t+1,1} \\ x_{t+1,2} \end{bmatrix} = 
  \begin{bmatrix} x_{t,1} \\ x_{t,2} \end{bmatrix} + 
  \mathrm{d}t \left(
    \begin{bmatrix} 0 & 1 \\ -\omega^2 & -2 \zeta \omega \end{bmatrix} 
    \begin{bmatrix} x_{t,1} \\ x_{t,2} \end{bmatrix} + 
    \begin{bmatrix} 0 \\ G \omega^2 \end{bmatrix} n_t
  \right),
  \qquad
  n_t \sim \mathcal{N}(0,1) ~~\forall t
$$

which can be considered as an Euler integration scheme applied to a state space formulation of a second-order linear system with a standard nomral noise input at each time step. As the 'energy' of the input noise process implicitly depends on the time step here, this introduces an implicit dependence on the amplitude of the coloured noise process on $\mathrm{dt}$. An alternative is to consider the noise process as being defined by a system of stochastic differential equations

$$
  \begin{bmatrix} \mathrm{d}x_{1}(t) \\ \mathrm{d}x_{2}(t) \end{bmatrix} = 
    \begin{bmatrix} 0 & 1 \\ -\omega^2 & -2 \zeta \omega \end{bmatrix} 
    \begin{bmatrix} x_{1}(t) \\ x_{2}(t) \end{bmatrix} \mathrm{d}t + 
    \begin{bmatrix} 0 \\ G \omega^2 \end{bmatrix} \mathrm{d}n(t)
$$

where $\mathrm{d}n(t)$ is a Gaussian white noise process with unit variance. Applying an Euler-Maruyama numerical integration scheme to this system of equations then gives an update of the form

$$
  \begin{bmatrix} x_{t+1,1} \\ x_{t+1,2} \end{bmatrix} = 
  \begin{bmatrix} x_{t,1} \\ x_{t,2} \end{bmatrix} + 
  \mathrm{d}t
    \begin{bmatrix} 0 & 1 \\ -\omega^2 & -2 \zeta \omega \end{bmatrix} 
    \begin{bmatrix} x_{t,1} \\ x_{t,2} \end{bmatrix} + 
  \sqrt{\mathrm{d}t}
    \begin{bmatrix} 0 \\ G \omega^2 \end{bmatrix} n_t,
  \qquad
  n_t \sim \mathcal{N}(0,1) ~~\forall t.
$$

It can be seen here the SDE based updates are equivalent to the Farrell et al. updates if we multiply $G$ by $\sqrt{\mathrm{d}t}$.

By default `pompy` uses the above SDE based formulation for the coloured noise process driving the velocity field, however updates consistent with the Farrell et al. (2002) implementation can be obtained by passsing a `use_original_noise_updates` argument with value `True` to the `WindModel` constructor. For consistency with the Farrell et al. (2002) formulation we do this here.

In [4]:
wind_model_params['use_original_noise_updates'] = True

In addition to the above scalar parameters, the wind model also requires specification of the extents of rectangular region in which to simulate the velocity field. Here we use a $100$ by $100\,\textrm{m}$ region as used in Farrell et al. with the origin centred in the $y$ direction.

In [5]:
wind_model_params['sim_region'] = models.Rectangle(
    x_min=0., y_min=-50., x_max=100., y_max=50.)

The wind model simulates the velocity field on a equispaced rectilinear grid over the simulation region using a finite difference method. Farrell et al. (2002) states that as the wind model is not intended to model the small-scale turbulent structure of the wind velocity field, a relatively coarse spatial discretisation with separation $\Delta \in [5,10]\,\textrm{m}$ between grid nodes is reasonable. Here we use a $21 \times 21$ grid, which given the $100\,\textrm{m}$ dimensions of the simulation region in both dimensions corresponds to a separation $\Delta = 5\,\textrm{m}$ between grid nodes. 

In [6]:
wind_model_params['n_x'] = 21  # Number of grid points in x-direction.
wind_model_params['n_y'] = 21  # Number of grid points in y-direction.

We now create wind model object using the defined parameters.

In [7]:
wind_model = models.WindModel(rng=rng, **wind_model_params)

## Plume model

The odour plume is modelled as a series of odour puffs which are released from a fixed source position. The odour puffs are dispersed by the modelled 2D wind velocity field plus a white noise process model of mid-scale puff mass diffusion relative to the plume centre line. The puffs also spread in size over time to model fine-scale diffusive processes.

The plume model is implemented in the `pompy.models.PlumeModel` class with the following correspondence between parameters in the `PlumeModel` constructor and the quantities defined in Farrel et al. (2002).

| `PlumeModel` parameter  | Farrell et al. symbol | Farrell et al. value       |
|-------------------------|-----------------------|----------------------------|
| `centre_rel_diff_scale` | $\sigma_v$            | 2 m / √s                   |
| `puff_release_rate`     | $n$                   | 10 puff / s                |
| `puff_init_rad`         | $R(0)$                | √0.001 m                   |
| `puff_spread_rate`      | $\gamma$              | 0.001 m² / s               |

In the cell below we define a dictionary of plume model parameters as in the above table.

In [8]:
plume_model_params = {
    'centre_rel_diff_scale': 2.,  # Scaling coefficient of diffusion of puffs
    'puff_release_rate': 10,  # Initial radius of the puffs
    'puff_init_rad': 0.001**0.5,  # Initial puff radius
    'puff_spread_rate': 0.001  # Rate of puff size spread over time
}

We define the plume simulation region using a `Rectangle` object as for the `WindModel` above. This is chosen as a subset of the wind simulation region to reduce the area we need to keep track of puffs within.

In [9]:
plume_model_params['sim_region'] = models.Rectangle(
    x_min=0., x_max=50., y_min=-12.5, y_max=12.5)

We additionally need to specify the position of the puff source for the simulation. Here we follow Farrell et al. (2002) by placing the source at the point with coordinates $x=5\textrm{m}$, $y=0\textrm{m}$ (in the coordinate system of the just defined simulation region) and with a zero height ($z$ coordinate).

In [10]:
plume_model_params['source_pos'] = (5., 0., 0.)

Finally we specify some additional settings to control the initial number of puffs released, maximum number of puffs to maintain in simulation (with puff generation halted once this value is reached) and whether to model dispersion of puffs in vertical ($z$ direction).

In [11]:
plume_model_params.update({
    'init_num_puffs': 10,
    'max_num_puffs': 1000,
    'model_z_disp': True,
})

We finally create plume model object using defined parameters and previously constructed wind model object.

In [12]:
plume_model = models.PlumeModel(
    rng=rng, wind_model=wind_model, **plume_model_params)

## Animating a plume simulation

The main user facing output of the `PlumeModel` class is the `puff_array` property. This is a shape `(n_puffs, 4)` NumPy array where `n_puffs` is the number of puffs currently present in the simulation region. Each row corresponds to the properties of a single puff in the order $x$-coordinate, $y$-coordinate, $z$-coordinate and squared radius.

`pompy` includes a number of classes in the `processors` module which allow post-processing of a `puff_array` to produce more directly useful values. Here we will use the `ConcentrationArrayGenerator` class which produces a NumPy array representing the odour concentration field in the $xy$ plane at a specified height under an assumption of a Gaussian concentration distribution for each puff.

We first define a dictionary of parameters for the `ConcentrationArrayGenerator` class, specifying the dimensions of the grid to sample concentration values on, the height to sample at and the assumed 'molecular' content / mass of each puff (conserved during the simulation, we use the value of $Q$ = 8.3 × 10⁸ molecules / puff from Farrell et al. (2002) here though for the purposes of visualisation, the scaling is arbitrary).

In [13]:
array_gen_params = {
    'array_z': 0.,  # Height on z-axis at which to calculate concentrations
    'n_x': 500,  # Number of grid points to sample at across x-dimension.
    'n_y': 250,  # Number of grid points to sample at across y-dimension.
    'puff_mol_amount': 8.3e8  # Molecular content of each puff
}

We now create a concentration array generator object using these parameters.

In [14]:
array_gen = processors.ConcentrationArrayGenerator(
    array_xy_region=plume_model.sim_region, **array_gen_params)

The `WindModel` and `PlumeModel` objects both define `update` methods which forward integrate the models by one time step. We first integrate the wind model forward 1000 time steps of $\textrm{d}t=0.01\textrm{s}$ in order to 'equilibrate' the wind velocity field to a state typical of its stationary regime (rather than the atypical initial constant initialisation)

In [15]:
# Simulation timestep
dt = 0.01

# Run wind model forward to equilibrate
for k in range(1000):
    wind_model.update(dt)

We now use the Matplotlib `animation` module to visualise the simulated concentration field (calculated using the concentration array generator object just constructed given the `puff_array` property of the `plume_model` after each series of integrator updates), interleaving series of calls to the model `update` methods with plotting of the generated concentration fields as images.

In [16]:
# Set up figure
fig = plt.figure(figsize=(12, 6))
ax = fig.add_axes([0., 0., 1., 1.])
ax.axis('off')

# Display initial concentration field as image
conc_array = array_gen.generate_single_array(plume_model.puff_array)
conc_im = ax.imshow(
    conc_array.T, extent=plume_model.sim_region, vmin=0., vmax=1e10, cmap='Reds')

# Define animation update function
def update(i):
    # Do 10 time steps per frame update
    for k in range(10):
        wind_model.update(dt)
        plume_model.update(dt)
    conc_array = array_gen.generate_single_array(
        plume_model.puff_array)
    conc_im.set_data(conc_array.T)
    return [conc_im]

# Animate plume concentration
anim = FuncAnimation(fig, update, frames=1200, repeat=False, blit=True, interval=50)
plt.close(fig)
# anim.save('plume-farrell.mp4', dpi=100, fps=20, extra_args=['-vcodec', 'libx264'])
anim